In [1]:
import os

In [2]:
%pwd

'd:\\End-to-End-Food-Image-Recognition\\research'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionconfig:
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir: Path

In [4]:
from cvclassifier.constants import *  # noqa: F403
from cvclassifier.utils.common import read_yaml ,create_directories  # noqa: F401


In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,  # noqa: F405
        params_filepath=PARAMS_FILE_PATH  # noqa: F405
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionconfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionconfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


In [6]:
import os  # noqa: F401, F811
import zipfile  # noqa: F401
import gdown  # noqa: F401
from cvclassifier import logger
from cvclassifier.utils.common import get_size  # noqa: F401


In [9]:
import os
import gdown
import zipfile
from cvclassifier import logger
from cvclassifier.entity.config_entity import DataIngestionConfig

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        """
        Fetch data from the URL and download it as a zip file.
        """
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs(os.path.dirname(zip_download_dir), exist_ok=True)

            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]  # Extract Google Drive file ID
            prefix = 'https://drive.google.com/uc?export=download&id='
            gdown.download(prefix + file_id, zip_download_dir, quiet=False, fuzzy=True)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
            return zip_download_dir

        except Exception as e:
            raise e

    def extract_zip_file(self):
        """
        Extracts the downloaded zip file into the specified directory.
        """
        try:
            unzip_path = self.config.unzip_dir
            os.makedirs(unzip_path, exist_ok=True)

            if not os.path.exists(self.config.local_data_file):
                raise FileNotFoundError(f"Zip file not found at {self.config.local_data_file}")

            logger.info(f"Extracting {self.config.local_data_file} to {unzip_path}")
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
            logger.info(f"Extraction completed to {unzip_path}")

        except Exception as e:
            raise e


In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-08-03 11:08:38,009: INFO: common: yaml file: D:\End-to-End-Food-Image-Recognition\config\config.yaml loaded successfully]
[2025-08-03 11:08:38,010: INFO: common: yaml file: D:\End-to-End-Food-Image-Recognition\params.yaml loaded successfully]
[2025-08-03 11:08:38,011: INFO: common: created directory at: artifacts]
[2025-08-03 11:08:38,012: INFO: common: created directory at: artifacts/data_ingestion]
[2025-08-03 11:08:38,012: INFO: 3818088007: Downloading data from https://drive.google.com/file/d/1a-VlYWGvp5_P1v9GmbEaHsDmdGrPXMhE/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?id=1a-VlYWGvp5_P1v9GmbEaHsDmdGrPXMhE
From (redirected): https://drive.google.com/uc?id=1a-VlYWGvp5_P1v9GmbEaHsDmdGrPXMhE&confirm=t&uuid=63c9e520-8a07-4727-ab2a-10cb8ce482d0
To: d:\End-to-End-Food-Image-Recognition\research\artifacts\data_ingestion\data.zip
100%|██████████| 5.01G/5.01G [03:23<00:00, 24.6MB/s]

[2025-08-03 11:12:04,786: INFO: 3818088007: Downloaded data from https://drive.google.com/file/d/1a-VlYWGvp5_P1v9GmbEaHsDmdGrPXMhE/view?usp=sharing into file artifacts/data_ingestion/data.zip]
[2025-08-03 11:12:04,787: INFO: 3818088007: Extracting artifacts/data_ingestion/data.zip to artifacts/data_ingestion]


[2025-08-03 11:13:01,144: INFO: 3818088007: Extraction completed to artifacts/data_ingestion]
